In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

d_train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
d_train = d_train[d_train.loc[:, 'Cover_Type'] != 5]

x = d_train.drop(columns = ['Id', 'Cover_Type', 'Soil_Type7', 'Soil_Type15'])
N_input = x.shape[1]
scaler = StandardScaler()
scaler.fit(x)

d_test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
x_test = d_test.drop(columns = ['Id','Soil_Type7', 'Soil_Type15'])
x_test2 = scaler.transform(x_test)

N_repeat = 20
for i in range(N_repeat):
    d_train_sample = d_train.sample(frac=0.4)
    x_train = d_train_sample.drop(columns = ['Id', 'Cover_Type', 'Soil_Type7', 'Soil_Type15'])
    x_train2 = scaler.transform(x_train)
    
    y_train = d_train_sample.loc[:, 'Cover_Type']
    y_train = [y-1 if y < 5 else 4 for y in y_train]
    y_train = to_categorical(y_train)

    N = 120
    model = Sequential()
    model.add(Dense(units=N, input_shape=(N_input,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=N))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=N))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=N))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    model.fit(x_train2, y_train, epochs=90, batch_size=2000, verbose=0)
    
    predicted = np.argmax(model.predict(x_test2), axis=1)
    d_test[str(i)] = [y+1 if y < 4 else 6 if elev < 3000 else 7 for y,elev in zip(predicted, x_test.loc[:, 'Elevation'])]

d_test['Cover_Type'] =  d_test.loc[:, [str(i) for i in range(N_repeat)]].mode(axis='columns').iloc[:, 0].astype('int')
d_test.loc[:, ['Id', 'Cover_Type']].to_csv('/kaggle/working/submission.csv', header=True, index=False)